<a href="https://colab.research.google.com/github/agungfirdaus717-ux/torentotgd/blob/main/SRTTranslatorBingtranslate(Fast).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SRT Translator (Bing Translate)

In [ ]:
# ============================
# Install & Import Library
# ============================
!pip install pysrt aiohttp nest_asyncio beautifulsoup4

import re
import pysrt
import asyncio
import aiohttp
import nest_asyncio
from bs4 import BeautifulSoup
nest_asyncio.apply()

import requests

# ============================
# Ambil IG Token & Daftar Bahasa Bing
# ============================
def get_bing_info():
    url = "https://www.bing.com/translator"
    r = requests.get(url)
    ig = re.search(r'IG:"([^"]+)"', r.text).group(1)

    # Parse daftar bahasa dari HTML
    soup = BeautifulSoup(r.text, "html.parser")
    options = soup.select("select#tta_tgtsl option")
    langs = {opt["value"]: opt.text for opt in options if opt.get("value")}
    return ig, langs

# ============================
# Async Function Translate
# ============================
async def bing_translate_async(session, text, to_lang="id", from_lang="auto", ig=None):
    api_url = "https://www.bing.com/ttranslatev3"
    payload = {"fromLang": from_lang, "to": to_lang, "text": text}
    headers = {"Referer": "https://www.bing.com/translator"}
    params = {"IG": ig, "IID": "translator.5028"}

    try:
        async with session.post(api_url, data=payload, headers=headers, params=params) as res:
            data = await res.json()
            return data[0]["translations"][0]["text"]
    except:
        return text  # fallback kalau error

async def translate_batch(texts, to_lang="id", ig=None):
    async with aiohttp.ClientSession() as session:
        tasks = [bing_translate_async(session, t, to_lang=to_lang, ig=ig) for t in texts]
        return await asyncio.gather(*tasks)

# ============================
# Translate Subtitle Async
# ============================
def translate_srt_async(input_file, output_file, to_lang="id", batch_size=50):
    subs = pysrt.open(input_file)
    texts = [sub.text.replace("\n", " [NEWLINE] ") for sub in subs]

    translated_texts = []
    loop = asyncio.get_event_loop()
    ig, langs = get_bing_info()

    for i in range(0, len(texts), batch_size):
        chunk = texts[i:i+batch_size]
        print(f"⚡ Translating lines {i+1}-{i+len(chunk)} ...")
        translated_chunk = loop.run_until_complete(translate_batch(chunk, to_lang=to_lang, ig=ig))
        translated_texts.extend(translated_chunk)

    translated_texts = [t.replace(" [NEWLINE] ", "\n") for t in translated_texts]

    for sub, t in zip(subs, translated_texts):
        sub.text = t

    subs.save(output_file, encoding="utf-8")
    print(f"✅ Selesai! Subtitle tersimpan di: {output_file}")

# ============================
# Contoh Pemakaian (Interaktif)
# ============================
from google.colab import files

print("Upload file subtitle (.srt)")
uploaded = files.upload()

# Ambil daftar bahasa dari Bing
ig, langs = get_bing_info()

print("\n🌍 Daftar bahasa yang tersedia di Bing Translator:")
for i, (code, name) in enumerate(langs.items(), start=1):
    print(f"{i}. {name} ({code})")

# Pilih bahasa target
choice = int(input("\n👉 Pilih nomor bahasa target: "))
target_code = list(langs.keys())[choice-1]
print(f"✅ Bahasa target dipilih: {langs[target_code]} ({target_code})")

# Proses translate
input_file = list(uploaded.keys())[0]
output_file = f"output_{target_code}.srt"

translate_srt_async(input_file, output_file, to_lang=target_code)

# Download hasil
files.download(output_file)
